# MATH 476 Project

In [33]:
import numpy as np
import pandas as pd
import json
import re

In [2]:
df = pd.read_csv("products.csv")
df.head()

,uuid,category,date-collected,product,product-details,query,store
0,d0349131-9f38-4774-952a-a6e54c0e9554,vegetables,2021-08-24,vegetables,"{""tcin"":{""S"":""47095644""},""pricing"":{""M"":{""curr...",vegetables,target
1,93fc6bce-7760-4ee0-b29c-29df2bcb1b50,vegetables,2021-08-24,vegetables,"{""images"":{""M"":{""image_keys"":{""L"":[{""S"":""v1628...",vegetables,noon
2,91c95b6f-9644-42b0-a54e-ec6e30eb03ca,fish_seafood,2021-08-23,seafood,"{""images"":{""M"":{""image_keys"":{""L"":[{""S"":""v1558...",seafood,noon
3,71cd340b-27f1-477b-a9c0-5f021bbf3b16,oils_fats,2021-08-24,butter,"{""tcin"":{""S"":""53930343""},""pricing"":{""M"":{""curr...",butter,target
4,f87fc9e2-a1c2-4728-97fb-a798aa96f4df,small_electric_household_appliances,2021-08-25,rice cooker,"{""sku"":{""S"":""131735446""},""pricing"":{""M"":{""disp...",rice cooker,walmart


In [3]:
df.groupby('store')[['uuid']].count()

,uuid
store,
noon,1389
target,670
walmart,1362


## Separating Stores

Only focusing on Noon, Target, and Walmart

In [15]:
noon = df[df['store'] == 'noon']
target = df[df['store'] == 'target']
walmart = df[df['store'] == 'walmart']

nprod = pd.json_normalize(noon['product-details'].apply(json.loads))
tprod = pd.json_normalize(target['product-details'].apply(json.loads))
wprod = pd.json_normalize(walmart['product-details'].apply(json.loads))

## Walmart Data Cleaning

In [21]:
wprod.columns

Index(['sku.S', 'pricing.M.displayCondition.NULL',
       'pricing.M.isClearance.BOOL', 'pricing.M.isRollback.BOOL',
       'pricing.M.unit.N', 'pricing.M.displayPrice.N',
       'pricing.M.salesQuantity.N', 'pricing.M.priceUnitOfMeasure.S',
       'pricing.M.displayUnitPrice.S', 'pricing.M.previousPrice.N',
       'pricing.M.list.N', 'pricing.M.salesUnitOfMeasure.S',
       'info.M.image.M.thumbnail.S', 'info.M.maxAllowed.N',
       'info.M.salesUnit.S', 'info.M.name.S', 'info.M.weightIncrement.N',
       'info.M.isSnapEligible.BOOL', 'info.M.averageWeight.NULL',
       'info.M.productUrl.S', 'info.M.type.S', 'pricing.M.displayCondition.S',
       'info.M.averageWeight.N', 'pricing.M.unit.NULL',
       'pricing.M.priceUnitOfMeasure.NULL',
       'pricing.M.salesUnitOfMeasure.NULL', 'pricing.M.displayUnitPrice.NULL',
       'info.M.label.M.id.S', 'info.M.label.M.text.S', 'info.M.adsMeta.L',
       'uuid.S', 'pricing.M.displayPrice.NULL', 'pricing.M.list.NULL'],
      dtype='object')

In [188]:
# Subset Walmart price data
wprices = wprod[['info.M.name.S', 'pricing.M.unit.N', 'pricing.M.displayPrice.N', 'pricing.M.salesQuantity.N', 'pricing.M.displayUnitPrice.S']]

# Rename columns
wprices = wprices.rename(
    columns={'pricing.M.unit.N':'unitPrice', 
             'pricing.M.displayPrice.N':'price', 
             'pricing.M.salesQuantity.N':'amount', 
             'pricing.M.displayUnitPrice.S':'displayedUnitPrice'
            })

# Convert displayedUnitPrice column values to string
wprices['displayedUnitPrice'] = wprices['displayedUnitPrice'].astype(str)
wprices['displayedUnitPrice'] = wprices.apply(lambda r: np.nan if r['displayedUnitPrice'] == 'nan' else r['displayedUnitPrice'], axis=1)
wprices[wprices['displayedUnitPrice'].isna()]


,info.M.name.S,unitPrice,price,amount,displayedUnitPrice
19,"Small Hass Avocados, 4-6 Count Bag",2.98,2.98,1,NaN
29,Plaza Provision Mazola Vegetable Plus Canola 40oz,4.28,4.28,1,NaN
73,Celery Stalk,1.28,1.28,1,NaN
82,"Hot House Cucumber, Fresh, 1 Each",1.38,1.38,1,NaN
89,"Hostess Donettes Powdered Mini Donuts 10.5 Oz,...",1.98,1.98,1,NaN
...,...,...,...,...,...
1227,Iceberg Lettuce,1.28,1.28,1,NaN
1231,"Freshness Guaranteed Everything French Bread, ...",1,1,1,NaN
1273,"Great Value Half-Length Spaghetti, 16 oz",NaN,0.78,1,NaN
1314,"Land O Lakes® Light Butter with Canola Oil, 8 ...",2.24,2.24,1,NaN


In [200]:
# RegEx to extract units
def extractUnit(r):
    if pd.isnull(r['displayedUnitPrice']):
        return np.nan
    return re.search('/(.*)\)$', str(r['displayedUnitPrice'])).group(1)

wprices['unit'] = wprices.apply(extractUnit, axis=1)
wprices

,info.M.name.S,unitPrice,price,amount,displayedUnitPrice,unit
0,"Ben's Original™ Ready Rice™, Jasmine, 8.5 oz. ...",0.233,1.98,8.5,(23.3 ¢/oz),oz
1,Malt-O-Meal Honey Nut Scooters® Breakfast Cere...,0.138,5.38,39,(13.8 ¢/oz),oz
2,"Producers Whole Unflavored Milk, 1 Quart",0.042,1.34,32,(4.2 ¢/fl oz),fl oz
3,"Beef Flanken Style Ribs, 0.41 - 1.08 lb",6.34,6.09,1.76,($6.34/lb),lb
4,"Beef Skirt Steak, 0.89 - 1.86 lb",8.42,11.87,1.72,($8.42/lb),lb
...,...,...,...,...,...,...
1357,"Gemstone Medley Baby Potatoes, 1.5 lb Bag",0.157,3.77,24,(15.7 ¢/oz),oz
1358,"Marketside French Baguette, 10oz",0.198,1.98,10,(19.8 ¢/oz),oz
1359,Sam's Choice Frozen Wild Caught Mahi-Mahi Fill...,0.493,5.92,12,(49.3 ¢/oz),oz
1360,"Crisco Pure Vegetable Oil, 48 fl oz",NaN,2.98,48,(6.2 ¢/fl oz),fl oz


In [203]:
wprices['unit'].unique()

array(['oz', 'fl oz', 'lb', 'ea', nan, 'OZ', 'g', 'lb.', 'cu ft',
       'FLUID OUNCE', 'LB'], dtype=object)